reference : https://pseudo-lab.github.io/Tutorial-Book/chapters/object-detection/Ch5-Faster-R-CNN.html

# package

In [ ]:
import time
import os
import copy

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
%matplotlib inline

import natsort
import random
import shutil

from bs4 import BeautifulSoup
from PIL import Image

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')          
else:
    device = torch.device('cpu')
    
print('device:', device)

device: cpu


# path 설정

# mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# config

In [ ]:
#데이터 분리가 되어있는 경우

dir_a = r"/data/train"   #edit
dir_i = r"드라이브 경로/images/train"  #edit
testdir_a = r"드라이브 경로/annotations/test"  #edit
testdir_i = r"드라이브 경로/images/test"  #edit

annotations = natsort.natsorted(os.listdir(dir_a))
images = natsort.natsorted(os.listdir(dir_i))
test_annotations = natsort.natsorted(os.listdir(testdir_a))
test_images = natsort.natsorted(os.listdir(testdir_i))

print(len(annotations))
print(len(images))
print(len(test_annotations))
print(len(test_images))

print(annotations)
print(images)
print(test_annotations)
print(test_images)

FileNotFoundError: ignored

In [ ]:
#데이터 분리를 수행하는 경우

annotations = r"드라이브 경로"   #labeling data
images = r"드라이브 경로"  #raw data

print(len(os.listdir(annotations)))
print(len(os.listdir(images)))

!mkdir test_images
!mkdir test_annotations

random.seed(1234)
idx = random.sample(range(1748), 500)

for img in np.array(sorted(os.listdir(images)))[idx]:
    shutil.move(images, '드라이브 경로/test_images/')

for annot in np.array(sorted(os.listdir('annotations')))[idx]:
    shutil.move(annotations, '드라이브 경로/test_annotations/')

test_annotations = r"드라이브 경로/test_images/"
test_images = r"드라이브 경로/test_annotations/"

print(len(os.listdir(test_annotations)))
print(len(os.listdir(test_images)))

# setup

# model

In [ ]:
def get_model_instance_segmentation(num_classes):
  
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [ ]:
model = get_model_instance_segmentation('num_classes에 해당하는 숫자 지정')  #edit

model.to(device)

# train

In [ ]:
num_epochs = #edit 해당하는 숫자 넣기
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)  #optimizer 종류, lr, momentum, weight_decay 모두 변경 가능

In [ ]:
def train(epoch, num_epochs, model, optimizer, scheduler):

  for epoch in range(num_epochs):
    start = time.time()
    model.train()
    i = 0    
    epoch_loss = 0
    batch_loss_list = []
    progress = ProgressMonitor(length=len(train_dataset)) #train_dataset raw images 담긴 폴더로

    for imgs, annotations in data_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs) #raw image 저장한 것 찾아서 마지막 imgs 대체
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations] #labeling data 저장한 것 찾아서 마지막 annotations 대체
        loss_dict = model(imgs, annotations) 
        losses = sum(loss for loss in loss_dict.values())        

        optimizer.zero_grad()
        losses.backward()
        optimizer.step() 
        epoch_loss += losses

        batch_loss_list.append(loss.item())
        progress.update(epoch, num_epochs, batch.shape[0], sum(batch_loss_list)/len(batch_loss_list) )

    if scheduler:
        scheduler.step()

    print(f'epoch : {epoch+1}, Loss : {epoch_loss}, time : {time.time() - start}')

In [ ]:
#torch.save(model.state_dict(),f'model_{num_epochs}.pt')
#model.load_state_dict(torch.load(f'model_{num_epochs}.pt'))

# evaluation

In [ ]:
#시험 데이터 하나에 대해 예측

def make_prediction(model, img, threshold):
    model.eval()
    preds = model(img)
    for id in range(len(preds)) :
        idx_list = []

        for idx, score in enumerate(preds[id]['scores']) :
            if score > threshold : 
                idx_list.append(idx)

        preds[id]['boxes'] = preds[id]['boxes'][idx_list]  #바운딩 박스 좌표
        preds[id]['labels'] = preds[id]['labels'][idx_list]  #클래스
        preds[id]['scores'] = preds[id]['scores'][idx_list]  #점수

    return preds

In [ ]:
with torch.no_grad(): 
    # 테스트셋 배치사이즈= 2
    for imgs, annotations in test_data_loader:
        imgs = list(img.to(device) for img in imgs)

        pred = make_prediction(model, imgs, 0.5) #0.5 이상인 신뢰도 값만 저장 >변경 가능
        print(pred)
        break

# 기타

In [ ]:
#시각화

_idx = 1
print("Target : ", annotations[_idx]['labels'])
plot_image_from_output(imgs[_idx], annotations[_idx])
print("Prediction : ", pred[_idx]['labels'])
plot_image_from_output(imgs[_idx], pred[_idx])

In [ ]:
#전체 시험 데이터에 대해 예측

from tqdm import tqdm

labels = []
preds_adj_all = [] #예측 결과 담을 list
annot_all = [] #label 담을 list

for im, annot in tqdm(test_data_loader, position = 0, leave = True):
    im = list(img.to(device) for img in im)
    #annot = [{k: v.to(device) for k, v in t.items()} for t in annot]

    for t in annot:
        labels += t['labels']

    with torch.no_grad():
        preds_adj = make_prediction(model, im, 0.5) #0.5 이상인 신뢰도 값만 저장 >변경 가능
        preds_adj = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in preds_adj]
        preds_adj_all.append(preds_adj)
        annot_all.append(annot)

In [ ]:
%cd Tutorial-Book-Utils/
import utils_ObjectDetection as utils

In [ ]:
sample_metrics = []
for batch_i in range(len(preds_adj_all)):
    sample_metrics += utils.get_batch_statistics(preds_adj_all[batch_i], annot_all[batch_i], iou_threshold=0.5) 

true_positives, pred_scores, pred_labels = [torch.cat(x, 0) for x in list(zip(*sample_metrics))]  # 배치가 전부 합쳐짐
precision, recall, AP, f1, ap_class = utils.ap_per_class(true_positives, pred_scores, pred_labels, torch.tensor(labels))
mAP = torch.mean(AP)
print(f'mAP : {mAP}')
print(f'AP : {AP}')